# BVEP

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from loss_cal.tasks.bvep import BVEP
import torch
import matplotlib as mpl
import matplotlib.pyplot as plt
from loss_cal.costs import StepCost_weighted
from loss_cal.predictor import train, build_nn

plot_config_file = "loss_cal/utils/.matplotlibrc"
task_name="bvep"

## Set Up

In [3]:
bvep = BVEP(action_type="binary", num_actions=2)
bvep.task_name

'bvep'

Prior and Simulator

In [4]:
prior = bvep.get_prior()
simulator = bvep.get_simulator()

In [5]:
bvep.prior_params

{'low': tensor([-5.0000,  0.1000, -5.0000,  0.0000]),
 'high': tensor([ 0., 50.,  0.,  5.])}

## Decision Task

In [6]:
param = 0
eta_crit = -2.05
costs = [1.0, 20.0]
cost_fn = StepCost_weighted(costs, threshold=eta_crit)


### Load Data

In [7]:
bvep.generate_data(ntrain=5_000, ntest=1000, nval=1000, show_progress=True)

/Users/mgorecki/Documents/projects/loss-calibration/code/loss-cal/loss_cal/tasks/bvep.py:108: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  Epileptor2Dmodel = numba.jit(Epileptor2Dmodel, nopython=False)
/Users/mgorecki/Documents/projects/loss-calibration/code/loss-cal/loss_cal/tasks/bvep.py:76: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "Epileptor2Dmodel" failed type inference due to: Unknown attribute 'astype' of type float32

File "loss_cal/tasks/bvep.py", line 79:
        def Epileptor2Dmodel(params, constants, sigma, dt, ts):
            <source elided>

            eta.astype(float)
            ^

During: typing

/Users/mgorecki/Documents/projects/loss-calibration/code/loss-cal/loss_cal/tasks/bvep.py:108: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  Epileptor2Dmodel = numba.jit(Epileptor2Dmodel, nopython=False)
/Users/mgorecki/Documents/projects/loss-calibration/code/loss-cal/loss_cal/tasks/bvep.py:76: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "Epileptor2Dmodel" failed type inference due to: Unknown attribute 'astype' of type float32

File "loss_cal/tasks/bvep.py", line 79:
        def Epileptor2Dmodel(params, constants, sigma, dt, ts):
            <source elided>

            eta.astype(float)
            ^

During: typing

/Users/mgorecki/Documents/projects/loss-calibration/code/loss-cal/loss_cal/tasks/bvep.py:76: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "Epileptor2Dmodel" failed type inference due to: Unknown attribute 'astype' of type float32

File "loss_cal/tasks/bvep.py", line 79:
        def Epileptor2Dmodel(params, constants, sigma, dt, ts):
            <source elided>

            eta.astype(float)
            ^

During: typing of get attribute at /Users/mgorecki/Documents/projects/loss-calibration/code/loss-cal/loss_cal/tasks/bvep.py (79)

File "loss_cal/tasks/bvep.py", line 79:
        def Epileptor2Dmodel(params, constants, sigma, dt, ts):
            <source elided>

            eta.astype(float)
            ^

  def Epileptor2Dmodel(params, constants, sigma, dt, ts):
/Users/mgorecki/Documents/projects/loss-calibration/code/loss-cal/loss_cal/tasks/bvep.py:76: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting

/Users/mgorecki/Documents/projects/loss-calibration/code/loss-cal/loss_cal/tasks/bvep.py:108: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  Epileptor2Dmodel = numba.jit(Epileptor2Dmodel, nopython=False)
/Users/mgorecki/Documents/projects/loss-calibration/code/loss-cal/loss_cal/tasks/bvep.py:76: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "Epileptor2Dmodel" failed type inference due to: Unknown attribute 'astype' of type float32

File "loss_cal/tasks/bvep.py", line 79:
        def Epileptor2Dmodel(params, constants, sigma, dt, ts):
            <source elided>

            eta.astype(float)
            ^

During: typing

/Users/mgorecki/Documents/projects/loss-calibration/code/loss-cal/loss_cal/tasks/bvep.py:76: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "Epileptor2Dmodel" failed type inference due to: Unknown attribute 'astype' of type float32

File "loss_cal/tasks/bvep.py", line 79:
        def Epileptor2Dmodel(params, constants, sigma, dt, ts):
            <source elided>

            eta.astype(float)
            ^

During: typing of get attribute at /Users/mgorecki/Documents/projects/loss-calibration/code/loss-cal/loss_cal/tasks/bvep.py (79)

File "loss_cal/tasks/bvep.py", line 79:
        def Epileptor2Dmodel(params, constants, sigma, dt, ts):
            <source elided>

            eta.astype(float)
            ^

  def Epileptor2Dmodel(params, constants, sigma, dt, ts):
/Users/mgorecki/Documents/projects/loss-calibration/code/loss-cal/loss_cal/tasks/bvep.py:76: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting

...

/Users/mgorecki/Documents/projects/loss-calibration/code/loss-cal/loss_cal/tasks/bvep.py:108: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  Epileptor2Dmodel = numba.jit(Epileptor2Dmodel, nopython=False)
/Users/mgorecki/Documents/projects/loss-calibration/code/loss-cal/loss_cal/tasks/bvep.py:76: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "Epileptor2Dmodel" failed type inference due to: Unknown attribute 'astype' of type float32

File "loss_cal/tasks/bvep.py", line 79:
        def Epileptor2Dmodel(params, constants, sigma, dt, ts):
            <source elided>

            eta.astype(float)
            ^

During: typing

/Users/mgorecki/Documents/projects/loss-calibration/code/loss-cal/loss_cal/tasks/bvep.py:108: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  Epileptor2Dmodel = numba.jit(Epileptor2Dmodel, nopython=False)
/Users/mgorecki/Documents/projects/loss-calibration/code/loss-cal/loss_cal/tasks/bvep.py:76: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "Epileptor2Dmodel" failed type inference due to: Unknown attribute 'astype' of type float32

File "loss_cal/tasks/bvep.py", line 79:
        def Epileptor2Dmodel(params, constants, sigma, dt, ts):
            <source elided>

            eta.astype(float)
            ^

During: typing

/Users/mgorecki/Documents/projects/loss-calibration/code/loss-cal/loss_cal/tasks/bvep.py:108: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  Epileptor2Dmodel = numba.jit(Epileptor2Dmodel, nopython=False)
/Users/mgorecki/Documents/projects/loss-calibration/code/loss-cal/loss_cal/tasks/bvep.py:76: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "Epileptor2Dmodel" failed type inference due to: Unknown attribute 'astype' of type float32

File "loss_cal/tasks/bvep.py", line 79:
        def Epileptor2Dmodel(params, constants, sigma, dt, ts):
            <source elided>

            eta.astype(float)
            ^

During: typing

/Users/mgorecki/Documents/projects/loss-calibration/code/loss-cal/loss_cal/tasks/bvep.py:108: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  Epileptor2Dmodel = numba.jit(Epileptor2Dmodel, nopython=False)
/Users/mgorecki/Documents/projects/loss-calibration/code/loss-cal/loss_cal/tasks/bvep.py:76: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "Epileptor2Dmodel" failed type inference due to: Unknown attribute 'astype' of type float32

File "loss_cal/tasks/bvep.py", line 79:
        def Epileptor2Dmodel(params, constants, sigma, dt, ts):
            <source elided>

            eta.astype(float)
            ^

During: typing

/Users/mgorecki/Documents/projects/loss-calibration/code/loss-cal/loss_cal/tasks/bvep.py:108: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  Epileptor2Dmodel = numba.jit(Epileptor2Dmodel, nopython=False)
/Users/mgorecki/Documents/projects/loss-calibration/code/loss-cal/loss_cal/tasks/bvep.py:76: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "Epileptor2Dmodel" failed type inference due to: Unknown attribute 'astype' of type float32

File "loss_cal/tasks/bvep.py", line 79:
        def Epileptor2Dmodel(params, constants, sigma, dt, ts):
            <source elided>

            eta.astype(float)
            ^

During: typing

In [ ]:
theta_train = torch.load(f'./data/bvep/Epileptor2D/theta_train.pt')[:, param:param+1]
x_train = torch.load(f'./data/bvep/Epileptor2D/x_train.pt')
theta_val = torch.load(f'./data/bvep/Epileptor2D/theta_val.pt')[:, param:param+1]
x_val = torch.load(f'./data/bvep/Epileptor2D/x_val.pt')
print(theta_train.shape, x_train.shape)

In [ ]:
# theta_train = torch.vstack([torch.load(f'./data/bvep/Epileptor6D/theta_{i}.pt') for i in range(1,6)])[:, param:param+1]
# x_train = torch.vstack([torch.load(f'./data/bvep/Epileptor6D/x_{i}.pt') for i in range(1,6)])
# theta_val = torch.load('./data/bvep/Epileptor6D/theta_0.pt')[:, param:param+1]
# x_val = torch.load('./data/bvep/Epileptor6D/x_0.pt')
# print(theta_train.shape, x_train.shape)

Remove invalid pbservations

In [ ]:
mask_train = torch.any(x_train.isnan(), dim=1) #NaN and inf
x_train = x_train[~mask_train]
theta_train = theta_train[~mask_train]

mask_val = torch.any(x_val.isnan(), dim=1)
x_val = x_val[~mask_val]
theta_val = theta_val[~mask_val]

print("train:", theta_train.shape, x_train.shape)
print("val", theta_val.shape, x_val.shape)

In [ ]:
x_train.isinf().any(), x_train.isnan().any()

Test cost fn

In [ ]:
n_train = 2500
theta_train = theta_train[:n_train]
x_train = x_train[:n_train]

In [ ]:
plt.hist(theta_train.numpy(), bins=50)
plt.show()

In [ ]:
actions_evaluate = bvep.actions.sample(x_train.shape[0])
plt.hist(cost_fn(theta_train, actions_evaluate).squeeze().numpy(), bins=20)
plt.show()

### Train NN

In [ ]:
nn = build_nn(
    model="fc",
    x_train=x_train,
    action_train=bvep.actions.sample(x_train.shape[0]),
    hidden_dims=[100, 100, 100, 100],
    output_dim=1,
    activation=torch.nn.Sigmoid(),
    output_transform=torch.nn.ReLU(),
)
print(nn, end="\n-----\n")

In [ ]:
nn, train_loss, val_loss = train(
    model=nn,
    x_train=x_train,
    theta_train=theta_train,
    cost_fn=cost_fn,
    x_val=x_val,
    theta_val=theta_val,
    actions=bvep.actions,
    learning_rate=1e-3,
    model_dir=f"./results/{task_name}/binary/notebook",
    stop_after_epochs=50,
    # max_num_epochs=10,
)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 2))
ax.plot(train_loss, label="train")
ax.plot(val_loss, label="val")
ax.set_title("Loss curve")
ax.set_xlabel("epochs")
ax.set_ylabel("loss")
ax.legend()
plt.show()

### Evaluate model

In [ ]:
predictions_train = nn(x_train, actions_evaluate)

In [ ]:
a = 0
sorted_order = torch.argsort(theta_train, dim=0).squeeze()
theta_train_sorted = theta_train[sorted_order]
actions_sorted = actions_evaluate[sorted_order]
cond =  actions_sorted == a
plt.plot(theta_train_sorted[cond].numpy().squeeze(), cost_fn(theta_train_sorted[cond], actions_sorted[cond]).detach().numpy().squeeze(), label="actual costs")
plt.plot(theta_train_sorted[cond].numpy().squeeze(), predictions_train[sorted_order][cond].detach().numpy().squeeze(), label="predicted costs")
plt.legend()
plt.title(f"a={a}")
plt.xlabel(r"$\theta$")
plt.ylabel("predicted costs")
plt.show()

In [ ]:
cost_fn(theta_train_sorted, actions_sorted).mean()

In [ ]:
predictions_train